In [ ]:
!mkdir data

In [ ]:
!pip install icrawler

In [ ]:
from icrawler.builtin import BingImageCrawler
import tensorflow as tf
import os

In [ ]:
classes = ['pepe meme','doge meme']
number = 100

In [ ]:
for c in classes:
  bing_crawler = BingImageCrawler(storage={'root_dir':f'data/{c.replace(" ","-")}'})
  bing_crawler.crawl(keyword=c,filters=None,max_num=number,offset=0)

2020-04-26 20:21:12,481 - INFO - icrawler.crawler - start crawling...
2020-04-26 20:21:12,482 - INFO - icrawler.crawler - starting 1 feeder threads...
2020-04-26 20:21:12,486 - INFO - feeder - thread feeder-001 exit
2020-04-26 20:21:12,486 - INFO - icrawler.crawler - starting 1 parser threads...
2020-04-26 20:21:12,493 - INFO - icrawler.crawler - starting 1 downloader threads...
2020-04-26 20:21:12,877 - INFO - parser - parsing result page https://www.bing.com/images/async?q=pepe meme&first=0
2020-04-26 20:21:12,987 - INFO - downloader - image #1	https://www.thewrap.com/wp-content/uploads/2016/09/1200.jpg
2020-04-26 20:21:13,009 - INFO - downloader - image #2	https://i.ytimg.com/vi/zvgkGewzUq4/maxresdefault.jpg
2020-04-26 20:21:13,072 - INFO - downloader - image #3	https://static.independent.co.uk/s3fs-public/thumbnails/image/2016/09/27/21/trump-pepe.jpg
2020-04-26 20:21:13,103 - INFO - downloader - image #4	https://upload.wikimedia.org/wikipedia/en/6/63/Feels_good_man.jpg
2020-04-26 2

In [ ]:
base_dir = 'data'

IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    shuffle=True,
    subset='training')



Found 100 images belonging to 2 classes.


In [ ]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(2, activation='softmax')
])


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_generator,epochs=3,steps_per_epoch=2)

Epoch 1/3
2/2 [==============================] - 1s 270ms/step - loss: 2.5229e-06 - accuracy: 1.0000
Epoch 2/3
2/2 [==============================] - 0s 185ms/step - loss: 5.4929e-05 - accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 0s 205ms/step - loss: 1.6583e-06 - accuracy: 1.0000


In [ ]:
saved_model_dir = 'save'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-04-26 20:41:56,164 - WARNING - tensorflow - From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: save/assets


2020-04-26 20:42:00,014 - INFO - tensorflow - Assets written to: save/assets
2020-04-26 20:42:10,140 - INFO - absl - Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False
